We need to create a simple pipeline to use for standardization of the dataset (that will be used later for standardisation of user inputs)

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pandas as pd


In [3]:
df = pd.read_csv("../raw_data/merged_country_level/merged_dataset_with_knn.csv").set_index("Unnamed: 0")
df.index.name = "country"

print(df.shape)
df.head()


(155, 5)


,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index
country,,,,,
afghanistan,960.545000,18.1,3.880000,24.9,24.24
albania,518.916429,22.2,81.410000,55.3,48.21
algeria,356.045500,22.8,16.540000,47.4,54.43
andorra,1257.460000,11.1,113.413333,84.7,75.56
angola,740.635000,27.1,22.910000,33.7,36.58


In [4]:
df.describe()

,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index
count,155.000000,155.000000,155.000000,155.000000,155.000000
mean,1085.577345,21.283656,99.077806,53.684301,58.613742
std,848.482198,6.835107,78.998202,15.264927,13.547936
min,227.355000,1.200000,3.230000,19.300000,23.860000
25%,579.861406,15.350000,38.450000,41.966667,48.550000
50%,807.235000,23.000000,81.780000,53.000000,59.380000
75%,1327.921731,27.000000,147.250000,66.050000,68.550000
max,6790.835000,32.100000,345.330000,84.700000,86.500000


In [5]:
#pipe with column transformer and minmaxscaler
from sklearn.compose import ColumnTransformer

# Define the column transformer with MinMaxScaler for all columns
column_transformer = ColumnTransformer(
    transformers=[
        ("minmax", MinMaxScaler(), df.columns)
    ]
)

# Create a new pipeline with the column transformer
pipeline = Pipeline(steps=[
    ("column_transformer", column_transformer)
])

# Fit the pipeline to the data
df_scaled = pipeline.fit_transform(df)

df_scaled = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)

df_scaled.head()

,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index
country,,,,,
afghanistan,0.111708,0.546926,0.001900,0.085627,0.006066
albania,0.044422,0.679612,0.228530,0.550459,0.388729
algeria,0.019607,0.699029,0.038907,0.429664,0.488027
andorra,0.156945,0.320388,0.322079,1.000000,0.825351
angola,0.078202,0.838188,0.057527,0.220183,0.203065


In [6]:
df_scaled.describe()

,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index
count,155.000000,155.000000,155.000000,155.000000,155.000000
mean,0.130757,0.649957,0.280175,0.525754,0.554817
std,0.129273,0.221201,0.230921,0.233409,0.216283
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.053707,0.457929,0.102952,0.346585,0.394157
50%,0.088349,0.705502,0.229611,0.515291,0.567050
75%,0.167680,0.834951,0.420988,0.714832,0.713442
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df_scaled.to_csv("../raw_data/merged_country_level/scaled_merged_data_after_imputation.csv")

In [8]:
import pickle
with open('../models/scaling_pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

### extra section: figure out how to scale and extra column

In [9]:
print(pipeline.named_steps.keys())  # This will show you the steps in your pipeline

dict_keys(['column_transformer'])


In [14]:
user_input_dict = {}
user_input_dict["max_monthly_budget"] = 1257.4
normalized_inputs = {}

In [ ]:
# Get the column transformer
column_transformer = pipeline.named_steps['column_transformer']
max_monthly_budget = user_input_dict["max_monthly_budget"]
        
# Transform the max monthly budget directly (it will apply the appropriate MinMaxScaler)
budget_transformed = column_transformer.transform([[max_monthly_budget, 0, 0, 0, 0]])[0][0]
normalized_inputs["max_monthly_budget"] = budget_transformed
normalized_inputs

ValueError: Specifying the columns using strings is only supported for dataframes.

In [ ]:

cost_scale = cost_scaler.scale_[0]  # The scale (std dev) for the first column (monthly cost)
cost_mean = cost_scaler.mean_[0]    # The mean for the first column (monthly cost)


# Apply the same scaling to max_monthly_budget
normalized_max_monthly_budget = (user_input_dict["max_monthly_budget"] - cost_mean) / cost_scale
normalized_inputs["max_monthly_budget"] = normalized_max_monthly_budget